In [1]:
import pandas as pd
import numpy as np

######## Data preproceesing ########

import matplotlib as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

####### Regression model #########

from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR 
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor

###### package for madel selection ####

from sklearn.model_selection import cross_validate,cross_val_score,RepeatedKFold
from sklearn.metrics import mean_squared_error,r2_score
from math import sqrt 

###### to ignore warnings ####

import warnings
warnings.filterwarnings('ignore')

#####  model delpoyment #####

import pickle



In [2]:
data=pd.read_csv('SecondCar.csv')
data.head()

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.0,106001,100000
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.0,108556,100000
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.0,120678,120000
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.0,122917,100000
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.0,144902,130000


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              4340 non-null   object 
 1   year              4340 non-null   int64  
 2   km_driven         4340 non-null   int64  
 3   fuel              4337 non-null   object 
 4   seller_type       4338 non-null   object 
 5   transmission      4338 non-null   object 
 6   owner             4337 non-null   object 
 7   Rating            4336 non-null   float64
 8   ExShowroom Price  4340 non-null   int64  
 9   selling_price     4340 non-null   int64  
dtypes: float64(1), int64(4), object(5)
memory usage: 339.2+ KB


# Misiing value Treatment

1.cheaking missing value count

In [4]:
data.isnull().sum()

name                0
year                0
km_driven           0
fuel                3
seller_type         2
transmission        2
owner               3
Rating              4
ExShowroom Price    0
selling_price       0
dtype: int64

In [5]:
data[data.isnull().any( axis=1)==True]

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price
352,Chevrolet Sail 1.2 Base,2015,35000,NaN,Individual,Manual,First Owner,NaN,300235,260000
492,Chevrolet Sail 1.2 Base,2015,35000,Petrol,Individual,Manual,NaN,NaN,328139,260000
670,Tata Indigo Grand Petrol,2014,60000,Petrol,NaN,Manual,Second Owner,15.00,365382,240000
1028,Maruti Wagon R VXI BS IV with ABS,2014,64000,Petrol,Individual,Manual,Second Owner,NaN,436092,290000
1139,Maruti Alto LX BSIII,2007,125000,Petrol,Individual,NaN,First Owner,9.92,453407,140000
1972,Maruti Wagon R LXI Minor,2007,50000,Petrol,NaN,Manual,First Owner,14.00,560844,135000
2731,Chevrolet Enjoy TCDi LTZ 7 Seater,2013,33000,NaN,Individual,Manual,Second Owner,NaN,744569,390000
3928,Hyundai Creta 1.6 VTVT S,2015,25000,Petrol,Individual,NaN,First Owner,12.00,1511680,850000
4068,Toyota Corolla Altis G AT,2016,50000,NaN,Individual,Automatic,First Owner,11.00,1794249,900000
4186,Toyota Corolla Altis 1.8 VL CVT,2018,25000,Petrol,Dealer,Automatic,NaN,11.13,2448803,1650000


2.according to column type : use median or mode for missing values

In [6]:
for col in data.columns:
    
    if data[col].dtype == 'int32' or data[col].dtype == 'int64' or data[col].dtype == 'float32' or data[col].dtype == 'float64':
        
        median = data[col].median()
        
        data[col].fillna(median, inplace = True)

    else:
        
        data = data.fillna(data.mode().iloc[0])
    


In [7]:
data[data.isnull().any(axis = 1) == True]

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price


# unique vulue cheak for deleting categorical columns

In [8]:
data.nunique()

name                1491
year                  27
km_driven            770
fuel                   5
seller_type            3
transmission           2
owner                  5
Rating               566
ExShowroom Price    4331
selling_price        445
dtype: int64

In [9]:
len(data)

4340

if unique value = 1 or unique value = len(data)

then delete that column

# Feature Engineering

In [10]:
data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.00,106001,100000
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.00,108556,100000
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.00,120678,120000
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.00,122917,100000
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.00,144902,130000
...,...,...,...,...,...,...,...,...,...,...
4335,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9407487,4950000
4336,BMW 5 Series 520d Luxury Line,2019,12999,Diesel,Dealer,Automatic,First Owner,9.71,9598350,4800000
4337,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9857238,4950000
4338,Audi RS7 2015-2019 Sportback Performance,2016,13000,Petrol,Dealer,Automatic,First Owner,14.09,14235729,8900000


In [11]:
data['name'].str.split(' ')

0                   [Tata, Indica, Vista, Aqua, 1.4, TDI]
1                                  [Tata, Nano, Lx, BSIV]
2                 [Tata, Indica, Vista, Aqua, TDI, BSIII]
3                          [Maruti, Wagon, R, LXI, Minor]
4                                   [Chevrolet, Beat, LT]
                              ...                        
4335                        [BMW, X5, xDrive, 30d, xLine]
4336                 [BMW, 5, Series, 520d, Luxury, Line]
4337                        [BMW, X5, xDrive, 30d, xLine]
4338       [Audi, RS7, 2015-2019, Sportback, Performance]
4339    [Mercedes-Benz, S-Class, S, 350d, Connoisseurs...
Name: name, Length: 4340, dtype: object

In [12]:
data['name'].str.split(' ').str[0]

0                Tata
1                Tata
2                Tata
3              Maruti
4           Chevrolet
            ...      
4335              BMW
4336              BMW
4337              BMW
4338             Audi
4339    Mercedes-Benz
Name: name, Length: 4340, dtype: object

In [13]:
print(list(data['name'].str.split(' ').str[0].unique()))

['Tata', 'Maruti', 'Chevrolet', 'Hyundai', 'Ford', 'Volkswagen', 'Mahindra', 'Fiat', 'Nissan', 'Renault', 'Toyota', 'Datsun', 'Honda', 'Skoda', 'Ambassador', 'OpelCorsa', 'Daewoo', 'Force', 'Mercedes-Benz', 'BMW', 'Audi', 'Mitsubishi', 'Jeep', 'Isuzu', 'Kia', 'Volvo', 'Jaguar', 'MG', 'Land']


In [14]:
data['name'].str.split(' ').str[0].nunique()

29

In [15]:
data['Company_Name']=data['name'].str.split(' ').str[0]
data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,Company_Name
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.00,106001,100000,Tata
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.00,108556,100000,Tata
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.00,120678,120000,Tata
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.00,122917,100000,Maruti
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.00,144902,130000,Chevrolet
...,...,...,...,...,...,...,...,...,...,...,...
4335,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9407487,4950000,BMW
4336,BMW 5 Series 520d Luxury Line,2019,12999,Diesel,Dealer,Automatic,First Owner,9.71,9598350,4800000,BMW
4337,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9857238,4950000,BMW
4338,Audi RS7 2015-2019 Sportback Performance,2016,13000,Petrol,Dealer,Automatic,First Owner,14.09,14235729,8900000,Audi


# creating feature from numerical column- ExShowroom price

In [16]:
round(data['ExShowroom Price'].describe())

count        4340.0
mean       845381.0
std        884841.0
min        106001.0
25%        445390.0
50%        596055.0
75%        946243.0
max      15538153.0
Name: ExShowroom Price, dtype: float64

In [17]:
label = ['Range1','Range2','Range3','Range4','Range5']
label

['Range1', 'Range2', 'Range3', 'Range4', 'Range5']

In [18]:
data['ExPriceRange']=pd.qcut(data['ExShowroom Price'],q=[0,0.50,0.75,0.90,0.98,1],labels=label)

data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,Company_Name,ExPriceRange
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.00,106001,100000,Tata,Range1
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.00,108556,100000,Tata,Range1
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.00,120678,120000,Tata,Range1
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.00,122917,100000,Maruti,Range1
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.00,144902,130000,Chevrolet,Range1
...,...,...,...,...,...,...,...,...,...,...,...,...
4335,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9407487,4950000,BMW,Range5
4336,BMW 5 Series 520d Luxury Line,2019,12999,Diesel,Dealer,Automatic,First Owner,9.71,9598350,4800000,BMW,Range5
4337,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9857238,4950000,BMW,Range5
4338,Audi RS7 2015-2019 Sportback Performance,2016,13000,Petrol,Dealer,Automatic,First Owner,14.09,14235729,8900000,Audi,Range5


# creating feature from numerical column- km_driven

In [19]:
round(data['km_driven'].describe())

count      4340.0
mean      66216.0
std       46644.0
min           1.0
25%       35000.0
50%       60000.0
75%       90000.0
max      806599.0
Name: km_driven, dtype: float64

In [20]:
km_label = ['km1','km2','km3','km4','km5']
km_label

['km1', 'km2', 'km3', 'km4', 'km5']

In [21]:
data['kmRange']=pd.qcut(data['km_driven'],q=[0,0.50,0.75,0.90,0.98,1],labels=km_label)
data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,Company_Name,ExPriceRange,kmRange
0,Tata Indica Vista Aqua 1.4 TDI,2010,120000,Diesel,Individual,Manual,Second Owner,9.00,106001,100000,Tata,Range1,km3
1,Tata Nano Lx BSIV,2012,50000,Petrol,Individual,Manual,Second Owner,9.00,108556,100000,Tata,Range1,km1
2,Tata Indica Vista Aqua TDI BSIII,2011,70000,Diesel,Individual,Manual,First Owner,11.00,120678,120000,Tata,Range1,km2
3,Maruti Wagon R LXI Minor,2010,80000,Petrol,Individual,Manual,Second Owner,9.00,122917,100000,Maruti,Range1,km2
4,Chevrolet Beat LT,2010,80000,Petrol,Individual,Manual,Second Owner,10.00,144902,130000,Chevrolet,Range1,km2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9407487,4950000,BMW,Range5,km1
4336,BMW 5 Series 520d Luxury Line,2019,12999,Diesel,Dealer,Automatic,First Owner,9.71,9598350,4800000,BMW,Range5,km1
4337,BMW X5 xDrive 30d xLine,2019,30000,Diesel,Dealer,Automatic,First Owner,11.00,9857238,4950000,BMW,Range5,km1
4338,Audi RS7 2015-2019 Sportback Performance,2016,13000,Petrol,Dealer,Automatic,First Owner,14.09,14235729,8900000,Audi,Range5,km1


# Lebal encoding

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4340 entries, 0 to 4339
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   name              4340 non-null   object  
 1   year              4340 non-null   int64   
 2   km_driven         4340 non-null   int64   
 3   fuel              4340 non-null   object  
 4   seller_type       4340 non-null   object  
 5   transmission      4340 non-null   object  
 6   owner             4340 non-null   object  
 7   Rating            4340 non-null   float64 
 8   ExShowroom Price  4340 non-null   int64   
 9   selling_price     4340 non-null   int64   
 10  Company_Name      4340 non-null   object  
 11  ExPriceRange      4340 non-null   category
 12  kmRange           4340 non-null   category
dtypes: category(2), float64(1), int64(4), object(6)
memory usage: 382.0+ KB


In [23]:
lb_en= LabelEncoder()

data['name']=lb_en.fit_transform(data['name'])
data['fuel']=lb_en.fit_transform(data['fuel'])
data['seller_type']=lb_en.fit_transform(data['seller_type'])
data['transmission']=lb_en.fit_transform(data['transmission'])
data['Company_Name']=lb_en.fit_transform(data['Company_Name'])
data['owner']=lb_en.fit_transform(data['owner'])
data['ExPriceRange']=lb_en.fit_transform(data['ExPriceRange'])
data['kmRange']=lb_en.fit_transform(data['kmRange'])

data

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,Company_Name,ExPriceRange,kmRange
0,1228,2010,120000,1,1,1,2,9.00,106001,100000,25,0,2
1,1283,2012,50000,4,1,1,2,9.00,108556,100000,25,0,0
2,1229,2011,70000,1,1,1,0,11.00,120678,120000,25,0,1
3,1041,2010,80000,4,1,1,2,9.00,122917,100000,18,0,1
4,56,2010,80000,4,1,1,2,10.00,144902,130000,3,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4335,39,2019,30000,1,0,0,0,11.00,9407487,4950000,2,4,0
4336,30,2019,12999,1,0,0,0,9.71,9598350,4800000,2,4,0
4337,39,2019,30000,1,0,0,0,11.00,9857238,4950000,2,4,0
4338,25,2016,13000,4,0,0,0,14.09,14235729,8900000,1,4,0


# Correlation range(-1 to +1)

In [24]:
corr =data.corr()

corr.style.background_gradient(cmap='coolwarm')

,name,year,km_driven,fuel,seller_type,transmission,owner,Rating,ExShowroom Price,selling_price,Company_Name,ExPriceRange,kmRange
name,1.000000,-0.051902,0.126203,-0.083850,0.140802,0.087785,0.032819,0.020311,-0.068402,-0.077598,0.974513,-0.050665,0.123186
year,-0.051902,1.000000,-0.419688,-0.120528,-0.098352,-0.143800,-0.414705,-0.003809,0.304871,0.413922,-0.039724,0.468329,-0.380108
km_driven,0.126203,-0.419688,1.000000,-0.285634,0.113689,0.120226,0.297115,-0.003163,-0.165105,-0.192289,0.131461,-0.205417,0.897639
fuel,-0.083850,-0.120528,-0.285634,1.000000,0.038387,0.040445,-0.010301,-0.012126,-0.234674,-0.269779,-0.109394,-0.319322,-0.288339
seller_type,0.140802,-0.098352,0.113689,0.038387,1.000000,0.174925,0.165681,-0.005921,-0.132745,-0.151554,0.144218,-0.123147,0.122044
transmission,0.087785,-0.143800,0.120226,0.040445,0.174925,1.000000,0.078893,0.024621,-0.516677,-0.530205,0.110699,-0.438358,0.100282
owner,0.032819,-0.414705,0.297115,-0.010301,0.165681,0.078893,1.000000,0.007293,-0.167726,-0.207840,0.036452,-0.244165,0.303189
Rating,0.020311,-0.003809,-0.003163,-0.012126,-0.005921,0.024621,0.007293,1.000000,-0.000455,-0.021932,0.023503,0.030101,-0.013790
ExShowroom Price,-0.068402,0.304871,-0.165105,-0.234674,-0.132745,-0.516677,-0.167726,-0.000455,1.000000,0.960629,-0.087559,0.772826,-0.139062
selling_price,-0.077598,0.413922,-0.192289,-0.269779,-0.151554,-0.530205,-0.207840,-0.021932,0.960629,1.000000,-0.096858,0.775333,-0.164481


In [25]:
data=data.drop(['name','km_driven','ExShowroom Price'],axis=1)
data.head()

,year,fuel,seller_type,transmission,owner,Rating,selling_price,Company_Name,ExPriceRange,kmRange
0,2010,1,1,1,2,9.0,100000,25,0,2
1,2012,4,1,1,2,9.0,100000,25,0,0
2,2011,1,1,1,0,11.0,120000,25,0,1
3,2010,4,1,1,2,9.0,100000,18,0,1
4,2010,4,1,1,2,10.0,130000,3,0,1


In [26]:
corr =data.corr()

corr.style.background_gradient(cmap='coolwarm')

,year,fuel,seller_type,transmission,owner,Rating,selling_price,Company_Name,ExPriceRange,kmRange
year,1.000000,-0.120528,-0.098352,-0.143800,-0.414705,-0.003809,0.413922,-0.039724,0.468329,-0.380108
fuel,-0.120528,1.000000,0.038387,0.040445,-0.010301,-0.012126,-0.269779,-0.109394,-0.319322,-0.288339
seller_type,-0.098352,0.038387,1.000000,0.174925,0.165681,-0.005921,-0.151554,0.144218,-0.123147,0.122044
transmission,-0.143800,0.040445,0.174925,1.000000,0.078893,0.024621,-0.530205,0.110699,-0.438358,0.100282
owner,-0.414705,-0.010301,0.165681,0.078893,1.000000,0.007293,-0.207840,0.036452,-0.244165,0.303189
Rating,-0.003809,-0.012126,-0.005921,0.024621,0.007293,1.000000,-0.021932,0.023503,0.030101,-0.013790
selling_price,0.413922,-0.269779,-0.151554,-0.530205,-0.207840,-0.021932,1.000000,-0.096858,0.775333,-0.164481
Company_Name,-0.039724,-0.109394,0.144218,0.110699,0.036452,0.023503,-0.096858,1.000000,-0.055800,0.136270
ExPriceRange,0.468329,-0.319322,-0.123147,-0.438358,-0.244165,0.030101,0.775333,-0.055800,1.000000,-0.182295
kmRange,-0.380108,-0.288339,0.122044,0.100282,0.303189,-0.013790,-0.164481,0.136270,-0.182295,1.000000


# Identify the x and y

In [27]:
x= data.drop('selling_price',axis=1)
y=data.selling_price

In [28]:
y

0        100000
1        100000
2        120000
3        100000
4        130000
         ...   
4335    4950000
4336    4800000
4337    4950000
4338    8900000
4339    8150000
Name: selling_price, Length: 4340, dtype: int64

In [29]:
x

,year,fuel,seller_type,transmission,owner,Rating,Company_Name,ExPriceRange,kmRange
0,2010,1,1,1,2,9.00,25,0,2
1,2012,4,1,1,2,9.00,25,0,0
2,2011,1,1,1,0,11.00,25,0,1
3,2010,4,1,1,2,9.00,18,0,1
4,2010,4,1,1,2,10.00,3,0,1
...,...,...,...,...,...,...,...,...,...
4335,2019,1,0,0,0,11.00,2,4,0
4336,2019,1,0,0,0,9.71,2,4,0
4337,2019,1,0,0,0,11.00,2,4,0
4338,2016,4,0,0,0,14.09,1,4,0


# Scaling Data

In [30]:
all_x=list(x.columns)
x[all_x]=x[all_x]/x[all_x].max()

x

,year,fuel,seller_type,transmission,owner,Rating,Company_Name,ExPriceRange,kmRange
0,0.995050,0.25,0.5,1.0,0.5,0.600000,0.892857,0.0,0.50
1,0.996040,1.00,0.5,1.0,0.5,0.600000,0.892857,0.0,0.00
2,0.995545,0.25,0.5,1.0,0.0,0.733333,0.892857,0.0,0.25
3,0.995050,1.00,0.5,1.0,0.5,0.600000,0.642857,0.0,0.25
4,0.995050,1.00,0.5,1.0,0.5,0.666667,0.107143,0.0,0.25
...,...,...,...,...,...,...,...,...,...
4335,0.999505,0.25,0.0,0.0,0.0,0.733333,0.071429,1.0,0.00
4336,0.999505,0.25,0.0,0.0,0.0,0.647333,0.071429,1.0,0.00
4337,0.999505,0.25,0.0,0.0,0.0,0.733333,0.071429,1.0,0.00
4338,0.998020,1.00,0.0,0.0,0.0,0.939333,0.035714,1.0,0.00


In [31]:
x.describe()

,year,fuel,seller_type,transmission,owner,Rating,Company_Name,ExPriceRange,kmRange
count,4340.000000,4340.000000,4340.000000,4340.000000,4340.000000,4340.000000,4340.000000,4340.000000,4340.000000
mean,0.996580,0.616935,0.397235,0.896774,0.205069,0.797532,0.555752,0.217512,0.193260
std,0.002087,0.377102,0.229314,0.304289,0.308373,0.127473,0.243221,0.266177,0.251735
min,0.986139,0.000000,0.000000,0.000000,0.000000,0.600000,0.000000,0.000000,0.000000
25%,0.995545,0.250000,0.500000,1.000000,0.000000,0.668000,0.357143,0.000000,0.000000
50%,0.997030,0.250000,0.500000,1.000000,0.000000,0.800000,0.642857,0.125000,0.000000
75%,0.998020,1.000000,0.500000,1.000000,0.500000,0.917333,0.642857,0.312500,0.250000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [32]:
x_train ,x_test,y_train,y_test = train_test_split(x,y,test_size =0.3,random_state=12345)



In [33]:
poly_reg = PolynomialFeatures(degree = 3, interaction_only = False, include_bias = False)

x_train = poly_reg.fit_transform(x_train)  # Created 54 columns from 9 columns
x_test = poly_reg.fit_transform(x_test)    # Created 54 columns from 9 columns


len(x_train[0])

219

In [34]:
len(x_test[0])

219

In [35]:
x_train

array([[0.99405941, 1.        , 0.5       , ..., 0.        , 0.        ,
        0.125     ],
       [0.99653465, 0.25      , 0.5       , ..., 0.        , 0.        ,
        0.015625  ],
       [0.99405941, 1.        , 0.5       , ..., 0.        , 0.        ,
        0.125     ],
       ...,
       [0.99851485, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.9950495 , 0.25      , 0.        , ..., 0.015625  , 0.015625  ,
        0.015625  ],
       [0.99752475, 0.25      , 0.5       , ..., 0.0625    , 0.03125   ,
        0.015625  ]])

# Data Preprocessing ends1

# Regression :-

# Method 1:-  Linear Regression

In [36]:
# Model Declaration #

LR = LinearRegression()

#  Model Training #

LR.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_LR=LR.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_LR)*100,2
     ),
     "%"
     )

Accuracy =  82.83 %


# Method 2:-Ridge Regression1

In [37]:
# Model Declaration #

RR = Ridge()

#  Model Training #

RR.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_RR=RR.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_RR)*100,2
     ),
     "%"
     )

Accuracy =  82.71 %


# Method 3:-Lasso Regression

In [38]:
# Model Declaration #

Lasso = Lasso()

#  Model Training #

Lasso.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_Lasso=Lasso.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_Lasso)*100,2
     ),
     "%"
     )

Accuracy =  83.06 %


# Method 4:-ElasticNet

In [39]:
# Model Declaration #

EN = ElasticNet()

#  Model Training #

EN.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_EN=EN.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_EN)*100,2
     ),
     "%"
     )

Accuracy =  57.51 %


# Method 5:- kNeighbor Regression

In [40]:
# Model Declaration #

KNR = KNeighborsRegressor(n_neighbors=2)

#  Model Training #

KNR.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_KNR=KNR.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_KNR)*100,2
     ),
     "%"
     )

Accuracy =  81.29 %


# Method 6:- Support vector Regression

In [41]:
# Model Declaration #

SVR = SVR()

#  Model Training #

SVR.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_SVR=SVR.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_SVR)*100,2
     ),
     "%"
     )

Accuracy =  -5.8 %


# Method 7:- Bagging Regression

In [42]:
# Model Declaration #

BR = BaggingRegressor()

#  Model Spliting #
cv = RepeatedKFold(n_splits=10,n_repeats=3,random_state=12345)

Score=cross_val_score(BR,x,y,scoring='r2', cv=cv ,n_jobs=1)


print("Accuracy = ",round(100*Score.max(),2),"%")

Accuracy =  95.8 %


In [43]:
# Model Declaration #

BR = BaggingRegressor()

#  Model Training #

BR.fit(x_train,y_train)

#  Predict values for x_test(i.e. 30% exam questions)

y_pred_BR=BR.predict(x_test)

print("Accuracy = ",
     round(
         r2_score(y_test,y_pred_BR)*100,2
     ),
     "%"
     )

Accuracy =  85.14 %


# Model Deployment

Step 1:- file creation

In [44]:
filename='Regression_model_File.sav'
pickle.dump(BR,open(file=filename,mode='wb'))

Step 2 :- using weight file 

In [45]:
model_load=pickle.load(open(file=filename,mode='rb'))
y_pred_final=model_load.predict(x_train[1000:1001,])

In [46]:
y_pred_final

array([307100.])